<a href="https://colab.research.google.com/github/MUbarak123-56/qiskit-work/blob/master/rnn/rnn_sales_exp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/Sales Data/

/content/drive/MyDrive/Sales Data


In [3]:
%cd Data

/content/drive/MyDrive/Sales Data/Data


In [4]:
%ls

holidays_events.csv   oil.csv                stores.csv  train.csv
holidays_updated.csv  sample_submission.csv  test.csv    transactions.csv


In [5]:
import pandas as pd

In [6]:
import torch
import seaborn as sns; sns.set_style("darkgrid")
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, mean_squared_log_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

In [7]:
%ls

holidays_events.csv   oil.csv                stores.csv  train.csv
holidays_updated.csv  sample_submission.csv  test.csv    transactions.csv


In [46]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
holiday = pd.read_csv("holidays_updated.csv")
oil = pd.read_csv( "oil.csv")
stores = pd.read_csv("stores.csv")
transactions = pd.read_csv("transactions.csv")

In [47]:
train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [48]:
train["family"].value_counts()

AUTOMOTIVE                    90936
HOME APPLIANCES               90936
SCHOOL AND OFFICE SUPPLIES    90936
PRODUCE                       90936
PREPARED FOODS                90936
POULTRY                       90936
PLAYERS AND ELECTRONICS       90936
PET SUPPLIES                  90936
PERSONAL CARE                 90936
MEATS                         90936
MAGAZINES                     90936
LIQUOR,WINE,BEER              90936
LINGERIE                      90936
LAWN AND GARDEN               90936
LADIESWEAR                    90936
HOME CARE                     90936
HOME AND KITCHEN II           90936
BABY CARE                     90936
HOME AND KITCHEN I            90936
HARDWARE                      90936
GROCERY II                    90936
GROCERY I                     90936
FROZEN FOODS                  90936
EGGS                          90936
DELI                          90936
DAIRY                         90936
CLEANING                      90936
CELEBRATION                 

In [49]:
stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [50]:
stores = stores.rename(columns={"type": "store_type"})

In [51]:
holiday.head()

,Additional_Local,Additional_National,Bridge_National,Event_National,Holiday_Local,Holiday_National,Holiday_Regional,Transfer_Local,Transfer_National,date
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2012-03-02
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2012-04-01
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2012-04-12
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2012-04-14
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2012-04-21


In [52]:
holiday = holiday.rename(columns={"type": "holiday_type"})

In [53]:
train_holiday = holiday[(holiday["date"] >= "2013-01-01") &  (holiday["date"] <= "2017-08-15")]

In [54]:
train_holiday.head()

,Additional_Local,Additional_National,Bridge_National,Event_National,Holiday_Local,Holiday_National,Holiday_Regional,Transfer_Local,Transfer_National,date
34,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2013-01-01
35,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2013-02-11
36,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2013-02-12
37,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2013-03-02
38,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2013-04-01


In [55]:
train_holiday.tail()

,Additional_Local,Additional_National,Bridge_National,Event_National,Holiday_Local,Holiday_National,Holiday_Regional,Transfer_Local,Transfer_National,date
272,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017-07-24
273,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017-07-25
274,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2017-08-05
275,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2017-08-11
276,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2017-08-15


In [56]:
transactions.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [57]:
oil.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [58]:
train = train.merge(stores, on="store_nbr", how = "outer")
train.shape

(3000888, 10)

In [59]:
train = train.merge(train_holiday, on="date", how = "left")
train.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,store_type,cluster,Additional_Local,Additional_National,Bridge_National,Event_National,Holiday_Local,Holiday_National,Holiday_Regional,Transfer_Local,Transfer_National
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [60]:
train.shape

(3000888, 19)

In [61]:
train.tail()

,id,date,store_nbr,family,sales,onpromotion,city,state,store_type,cluster,Additional_Local,Additional_National,Bridge_National,Event_National,Holiday_Local,Holiday_National,Holiday_Regional,Transfer_Local,Transfer_National
3000883,3000883,2017-08-15,9,POULTRY,438.133,0,Quito,Pichincha,B,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,Quito,Pichincha,B,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148,Quito,Pichincha,B,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Quito,Pichincha,B,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3000887,3000887,2017-08-15,9,SEAFOOD,16.000,0,Quito,Pichincha,B,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [62]:
train["id"].value_counts()[:5]

0          1
3000286    1
3000288    1
3000289    1
3000290    1
Name: id, dtype: int64

In [63]:
train[train["id"]==961564]

,id,date,store_nbr,family,sales,onpromotion,city,state,store_type,cluster,Additional_Local,Additional_National,Bridge_National,Event_National,Holiday_Local,Holiday_National,Holiday_Regional,Transfer_Local,Transfer_National
1796101,961564,2014-06-25,39,EGGS,173.0,1,Cuenca,Azuay,B,6,0.0,0.0,0.0,1.0,2.0,0.0,1.0,0.0,0.0


In [64]:
train[train["id"] == 962196]

,id,date,store_nbr,family,sales,onpromotion,city,state,store_type,cluster,Additional_Local,Additional_National,Bridge_National,Event_National,Holiday_Local,Holiday_National,Holiday_Regional,Transfer_Local,Transfer_National
2851974,962196,2014-06-25,7,HOME AND KITCHEN I,0.0,0,Quito,Pichincha,D,8,0.0,0.0,0.0,1.0,2.0,0.0,1.0,0.0,0.0


In [65]:
train = train.merge(oil, on="date", how = "left")
train = train.merge(transactions, on=["date", "store_nbr"], how = "left")
train.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,store_type,cluster,...,Additional_National,Bridge_National,Event_National,Holiday_Local,Holiday_National,Holiday_Regional,Transfer_Local,Transfer_National,dcoilwtico,transactions
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN


In [66]:
train.shape

(3000888, 21)

In [67]:
new_train = train.copy()

In [68]:
new_train = new_train.sort_values("date").set_index("date")

In [70]:
new_train.head()

,id,store_nbr,family,sales,onpromotion,city,state,store_type,cluster,Additional_Local,Additional_National,Bridge_National,Event_National,Holiday_Local,Holiday_National,Holiday_Regional,Transfer_Local,Transfer_National,dcoilwtico,transactions
date,,,,,,,,,,,,,,,,,,,,
2013-01-01,0,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN
2013-01-01,1238,43,HOME APPLIANCES,0.0,0,Esmeraldas,Esmeraldas,E,10,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN
2013-01-01,1237,43,HOME AND KITCHEN II,0.0,0,Esmeraldas,Esmeraldas,E,10,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN
2013-01-01,1236,43,HOME AND KITCHEN I,0.0,0,Esmeraldas,Esmeraldas,E,10,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN
2013-01-01,1235,43,HARDWARE,0.0,0,Esmeraldas,Esmeraldas,E,10,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN


In [ ]:
#train_dates, val_dates, test_dates = new_train[:"2017-01"], new_train["2017-01": "2017-05"], new_train["2017-05":]

In [71]:
#train_dates

In [72]:
#val_dates

In [73]:
#test_dates

In [74]:
test_holiday = holiday[holiday["date"] > "2017-08-15"]

In [75]:
test_holiday.head()

,Additional_Local,Additional_National,Bridge_National,Event_National,Holiday_Local,Holiday_National,Holiday_Regional,Transfer_Local,Transfer_National,date
277,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2017-08-24
278,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2017-09-29
279,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2017-10-07
280,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2017-10-09
281,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2017-11-02


In [76]:
test.head()

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


In [77]:
test.shape

(28512, 5)

In [78]:
test = test.merge(stores, on="store_nbr", how = "left")
test.shape

(28512, 9)

In [79]:
test = test.merge(test_holiday, on="date", how = "left")
test.shape

(28512, 18)

In [80]:
test = test.merge(oil, on="date", how = "left")
test.shape

(28512, 19)

In [81]:
test = test.merge(transactions, on=["date", "store_nbr"], how = "left")
test.shape

(28512, 20)

In [82]:
test.head()

,id,date,store_nbr,family,onpromotion,city,state,store_type,cluster,Additional_Local,Additional_National,Bridge_National,Event_National,Holiday_Local,Holiday_National,Holiday_Regional,Transfer_Local,Transfer_National,dcoilwtico,transactions
0,3000888,2017-08-16,1,AUTOMOTIVE,0,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46.8,NaN
1,3000889,2017-08-16,1,BABY CARE,0,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46.8,NaN
2,3000890,2017-08-16,1,BEAUTY,2,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46.8,NaN
3,3000891,2017-08-16,1,BEVERAGES,20,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46.8,NaN
4,3000892,2017-08-16,1,BOOKS,0,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46.8,NaN


In [83]:
train.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,store_type,cluster,...,Additional_National,Bridge_National,Event_National,Holiday_Local,Holiday_National,Holiday_Regional,Transfer_Local,Transfer_National,dcoilwtico,transactions
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN


In [84]:
train.shape

(3000888, 21)

In [86]:
train.to_csv("train_merged.csv", index = False)
test.to_csv("test_merged.csv", index = False)

In [87]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3000888 entries, 0 to 3000887
Data columns (total 21 columns):
 #   Column               Dtype  
---  ------               -----  
 0   id                   int64  
 1   date                 object 
 2   store_nbr            int64  
 3   family               object 
 4   sales                float64
 5   onpromotion          int64  
 6   city                 object 
 7   state                object 
 8   store_type           object 
 9   cluster              int64  
 10  Additional_Local     float64
 11  Additional_National  float64
 12  Bridge_National      float64
 13  Event_National       float64
 14  Holiday_Local        float64
 15  Holiday_National     float64
 16  Holiday_Regional     float64
 17  Transfer_Local       float64
 18  Transfer_National    float64
 19  dcoilwtico           float64
 20  transactions         float64
dtypes: float64(12), int64(4), object(5)
memory usage: 503.7+ MB


In [88]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28512 entries, 0 to 28511
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   28512 non-null  int64  
 1   date                 28512 non-null  object 
 2   store_nbr            28512 non-null  int64  
 3   family               28512 non-null  object 
 4   onpromotion          28512 non-null  int64  
 5   city                 28512 non-null  object 
 6   state                28512 non-null  object 
 7   store_type           28512 non-null  object 
 8   cluster              28512 non-null  int64  
 9   Additional_Local     1782 non-null   float64
 10  Additional_National  1782 non-null   float64
 11  Bridge_National      1782 non-null   float64
 12  Event_National       1782 non-null   float64
 13  Holiday_Local        1782 non-null   float64
 14  Holiday_National     1782 non-null   float64
 15  Holiday_Regional     1782 non-null  